# Information

## CC2CAll_min_mrg 

This code uses:
+ Cleaner_Centroid with 2 points
+ Then uses the centroid to compare distance between all points, and calculates the distance.


In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy import stats  # For mode calculation

In [2]:
def detect_interactions(distances, min_consecutive=60, threshold=250, margin=30):
    """
    Detect interactions in a sequence of distances.

    Parameters:
        distances (list or numpy array): A sequence of distances.
        min_consecutive (int): Minimum consecutive frames required for an interaction.
        threshold (float): Threshold value to determine an interaction.
        margin (int): Number of frames where the rules don't apply and gaps can be merged.

    Returns:
        List of tuples where each tuple represents an interaction (start_frame, end_frame).
    """
    interactions = []
    current_start = None
    last_interaction_end = None  # Keep track of the end frame of the last interaction

    for i, distance in enumerate(distances):
        if distance < threshold:
            if current_start is None:
                current_start = i
        else:
            if current_start is not None:
                if i - current_start >= min_consecutive:
                    # Check if there is a margin to merge nearby interactions
                    if last_interaction_end is not None and current_start - last_interaction_end <= margin:
                        interactions[-1] = (interactions[-1][0], i - 1)
                    else:
                        interactions.append((current_start, i - 1))
                    last_interaction_end = i - 1
                current_start = None

    # Check if an interaction is ongoing at the end of the sequence
    if current_start is not None and len(distances) - current_start >= min_consecutive:
        interactions.append((current_start, len(distances) - 1))

    return interactions

In [3]:
directory = r"C:\Users\raulo\Desktop\ze_vids\phase_3\Tracks+Label_Top&Bot"
files = os.listdir(directory)

result_dataframes = {}

for file in files:
    df = pd.read_csv(os.path.join(directory, file))
    
    df["Behavior"].replace({"TS": "interaction", "jolts": "interaction"}, inplace=True)
    # Create a new DataFrame to store the results
    new_df = pd.DataFrame()

    # Add relevant data to the new DataFrame
    new_df["Behavior"] = df['Behavior']

    new_df['Client_Mouth_X_top'] = df['Client_Mouth']
    new_df['Client_Mouth_Y_top'] = df['Client_Mouth.1']
    new_df['Client_Mouth_Z_front']= df['Client_Mouth_Front.1']

    new_df['Client_S1_X_top'] = df['Client_Spine_1']
    new_df['Client_S1_Y_top'] = df['Client_Spine_1.1']
    new_df['Client_S1_Z_front']= df['Client_Spine_head_Front.1']
    
    new_df['Client_STop1_X_top'] = df['Client_Spine_1']
    new_df['Client_STop1_Y_top'] = df['Client_Spine_1.1']
    new_df['Client_STop1_Z_front']= df['Client_Body_top1_Front.1']
    
    new_df['Client_SBot1_X_top'] = df['Client_Spine_1']
    new_df['Client_SBot1_Y_top'] = df['Client_Spine_1.1']
    new_df['Client_SBot1_Z_front']= df['Client_Body_bot1_Front.1']

    new_df['Client_S2_X_top'] = df['Client_Spine_2']
    new_df['Client_S2_Y_top'] = df['Client_Spine_2.1']
    new_df['Client_S2_Z_front']= df['Client_Spine_mid_Front.1']
    
    new_df['Client_STop2_X_top'] = df['Client_Spine_2']
    new_df['Client_STop2_Y_top'] = df['Client_Spine_2.1']
    new_df['Client_STop2_Z_front']= df['Client_Body_top2_Front.1']
    
    new_df['Client_SBot2_X_top'] = df['Client_Spine_2']
    new_df['Client_SBot2_Y_top'] = df['Client_Spine_2.1']
    new_df['Client_SBot2_Z_front']= df['Client_Body_bot2_Front.1']
    
    new_df['Client_Tail_X_top'] = df['Client_Tail']
    new_df['Client_Tail_Y_top'] = df['Client_Tail.1']
    new_df['Client_Tail_Z_front']= df['Client_Tail_Front.1']
    
    new_df['Client_TailTipTop_X_top'] = df['Client_TailTip']
    new_df['Client_TailTipTop_Y_top'] = df['Client_TailTip.1']
    new_df['Client_TailTipTop_Z_front']= df['Client_Tail_Top_Front.1']

    new_df['Client_TailTipBot_X_top'] = df['Client_TailTip']
    new_df['Client_TailTipBot_Y_top'] = df['Client_TailTip.1']
    new_df['Client_TailTipBot_Z_front']= df['Client_Tail_Bot_Front.1']

    
    
    new_df['Cleaner_Mouth_X_top'] = df['Cleaner_Mouth']
    new_df['Cleaner_Mouth_Y_top'] = df['Cleaner_Mouth.1']
    new_df['Cleaner_Mouth_Z_front']= df['Cleaner_Mouth_Front.1']

    new_df['Cleaner_Spine1_X_top'] = df['Cleaner_Spine1']
    new_df['Cleaner_Spine1_Y_top'] = df['Cleaner_Spine1.1']
    new_df['Cleaner_Spine1_Z_front']= df['Cleaner_Spine1_Front.1']

    

    new_df["Frame"] = df['Frame']
    
    # Create a new column in new_df to store the centroid of the cleaner fish for each row
    new_df['Cleaner_Centroid_X'] = new_df[['Cleaner_Mouth_X_top', 'Cleaner_Spine1_X_top']].mean(axis=1)
    new_df['Cleaner_Centroid_Y'] = new_df[['Cleaner_Mouth_Y_top', 'Cleaner_Spine1_Y_top']].mean(axis=1)
    new_df['Cleaner_Centroid_Z'] = new_df[['Cleaner_Mouth_Z_front', 'Cleaner_Spine1_Z_front']].mean(axis=1)

    # Initialize a column to store the smallest distance for each row
    new_df['Min_Distance'] = np.inf  # Initialize with infinity

    # Loop through client points and calculate distance, updating 'Min_Distance' if a smaller distance is found
    for client_point in ['Client_Mouth', 'Client_S1', 'Client_STop1', 'Client_SBot1', 
                         'Client_S2', 'Client_STop2', 'Client_SBot2', 'Client_Tail', 'Client_TailTipTop', 'Client_TailTipBot']:
        client_x = new_df[f'{client_point}_X_top']
        client_y = new_df[f'{client_point}_Y_top']
        client_z = new_df[f'{client_point}_Z_front']

        distance = np.sqrt(
            (client_x - new_df['Cleaner_Centroid_X'])**2 +
            (client_y - new_df['Cleaner_Centroid_Y'])**2 +
            (client_z - new_df['Cleaner_Centroid_Z'])**2
        )

        new_df['Min_Distance'] = np.minimum(new_df['Min_Distance'], distance)
    
    new_df["Interaction_Predictions"] = "background"  # Initialization of predictions

    result_dataframes[file.split("_")[0]] = new_df

In [4]:
# Create a dictionary to store summary values for each DataFrame
summary_values = {}

for data_frame in result_dataframes:
    total_frames = 0
    correctly_detected_event = 0
    total = 0
    correctly_frame_count = 0

    interactions = detect_interactions(result_dataframes[data_frame]["Min_Distance"])

    for interaction in interactions:

        start_frame, end_frame = interaction

        if end_frame - start_frame + 1 >= 15:
            total += 1  # Increment the total count for eligible interactions
            total_frames += end_frame - start_frame + 1
            selected_data = result_dataframes[data_frame].loc[start_frame:end_frame, "Behavior"]
            mode_value = stats.mode(selected_data).mode[0]

            if mode_value == "interaction" or mode_value == "TS":
                correctly_detected_event +=1
                correctly_frame_count += end_frame - start_frame + 1

            # Store the prediction in the new column
            result_dataframes[data_frame].loc[start_frame:end_frame, "Interaction_Predictions"] = mode_value
                  
            # Store the summary values in the summary_values dictionary
            summary_values[data_frame] = {
            "total_frames": total_frames,
            "correctly_detected_event": correctly_detected_event,
            "total": total,
            "correctly_frame_count": correctly_frame_count
            }
            
#             if total == 1:
#                 print(f"\nData from: {data_frame}")            
#             print(f"Interaction detected from frame {start_frame} to {end_frame} (Duration: {end_frame - start_frame + 1} frames) with a true Behavior of {mode_value}")

C:\Users\raulo\AppData\Local\Temp\ipykernel_9400\828851113.py:20: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode_value = stats.mode(selected_data).mode[0]
C:\Users\raulo\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\raulo\AppData\Local\Temp\ipykernel_9400\828851113.py:20: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see ht

In [5]:
# Define a list of threshold and min_consecutive values to test
threshold_values = [100, 125, 150, 175, 200, 225, 250]  # Add more values if needed
min_consecutive_values = [30, 45, 60]  # Add more values if needed


for threshold in threshold_values:
    for min_consecutive in min_consecutive_values:

        summary_values = {}

        for data_frame_name, data_frame in result_dataframes.items():
            # Initialize variables to keep track of the current group
            current_group = None
            interaction_count = 0

            # Iterate through the "Behavior" column
            for behavior in data_frame["Behavior"]:
                if behavior == "interaction":
                    if current_group != "interaction":
                        # Start of a new interaction group
                        interaction_count += 1
                        current_group = "interaction"
                else:
                    current_group = None  # Reset the group if behavior is not "interaction"

            summary_values[data_frame_name] = {"true interaction count": interaction_count}

            
            # Initialize the summary values with default values
            summary_values[data_frame_name].update({
                "total_frames": 0,
                "correctly_detected_event": 0,
                "total": 0,
                "correctly_frame_count": 0
            })
            
            # Reset the other summary values to zero to avoid overwriting
            total_frames = 0
            correctly_detected_event = 0
            total = 0
            correctly_frame_count = 0

            interactions = detect_interactions(result_dataframes[data_frame_name]["Min_Distance"], min_consecutive, threshold)

            for interaction in interactions:
                start_frame, end_frame = interaction

                if end_frame - start_frame + 1 >= 15:
                    total += 1  # Increment the total count for eligible interactions
                    total_frames += end_frame - start_frame + 1
                    selected_data = result_dataframes[data_frame_name].loc[start_frame:end_frame, "Behavior"]
                    #mode_value = stats.mode(selected_data).mode[0]
                    mode_value = selected_data.mode().iloc[0]
                    
                    if mode_value == "interaction" or mode_value == "TS":
                        correctly_detected_event += 1
                        correctly_frame_count += end_frame - start_frame + 1

                    # Store the prediction in the new column
                    result_dataframes[data_frame_name].loc[start_frame:end_frame, "Interaction_Predictions"] = mode_value

                # Update the existing summary values in the summary_values dictionary
                summary_values[data_frame_name].update({
                "total_frames": total_frames,
                "correctly_detected_event": correctly_detected_event,
                "total": total,
                "correctly_frame_count": correctly_frame_count
                })

        #         if total == 1:
        #             print(f"\nData from: {data_frame_name}")
        #         print(f"Interaction detected from frame {start_frame} to {end_frame} (Duration: {end_frame - start_frame + 1} frames) with a true Behavior of {mode_value}")

        for data_frame, summary in summary_values.items():
            true_interaction_count = summary['true interaction count']
            total_frames = summary['total_frames']
            correctly_frame_count = summary['correctly_frame_count']
            total_detected_events = summary['total']
            correctly_detected_events = summary['correctly_detected_event']

            accuracy = (correctly_frame_count / total_frames) * 100 if total_frames != 0 else 0

            print(f"Data from: {data_frame} with threshold of: {threshold} and min_consecutives of: {min_consecutive}")
            print(f"This data has {true_interaction_count} true events")
            print(f"There were {correctly_frame_count} frames correctly classified, out of a total of {total_frames} frames classified as interaction.")
            print(f"This results in an accuracy of {accuracy:.2f}%.")
            print(f"There were a total of {correctly_detected_events} events detected correctly, out of {total_detected_events} total events detected. ({correctly_detected_events/total_detected_events *100:.2f}%)" 
                 if total_detected_events != 0 else "No events detected.")
            print(f"There was {correctly_detected_events/true_interaction_count * 100:.2f}% events detected")
            print("")

Data from: LD03 with threshold of: 100 and min_consecutives of: 30
This data has 69 true events
There were 5281 frames correctly classified, out of a total of 5976 frames classified as interaction.
This results in an accuracy of 88.37%.
There were a total of 56 events detected correctly, out of 68 total events detected. (82.35%)
There was 81.16% events detected

Data from: LD04 with threshold of: 100 and min_consecutives of: 30
This data has 63 true events
There were 7625 frames correctly classified, out of a total of 8579 frames classified as interaction.
This results in an accuracy of 88.88%.
There were a total of 65 events detected correctly, out of 82 total events detected. (79.27%)
There was 103.17% events detected

Data from: LD13 with threshold of: 100 and min_consecutives of: 30
This data has 81 true events
There were 3849 frames correctly classified, out of a total of 5121 frames classified as interaction.
This results in an accuracy of 75.16%.
There were a total of 62 events 

Data from: LD03 with threshold of: 125 and min_consecutives of: 45
This data has 69 true events
There were 6708 frames correctly classified, out of a total of 7441 frames classified as interaction.
This results in an accuracy of 90.15%.
There were a total of 55 events detected correctly, out of 65 total events detected. (84.62%)
There was 79.71% events detected

Data from: LD04 with threshold of: 125 and min_consecutives of: 45
This data has 63 true events
There were 8500 frames correctly classified, out of a total of 9335 frames classified as interaction.
This results in an accuracy of 91.06%.
There were a total of 61 events detected correctly, out of 72 total events detected. (84.72%)
There was 96.83% events detected

Data from: LD13 with threshold of: 125 and min_consecutives of: 45
This data has 81 true events
There were 5453 frames correctly classified, out of a total of 7076 frames classified as interaction.
This results in an accuracy of 77.06%.
There were a total of 63 events d

Data from: LD03 with threshold of: 150 and min_consecutives of: 60
This data has 69 true events
There were 8016 frames correctly classified, out of a total of 8917 frames classified as interaction.
This results in an accuracy of 89.90%.
There were a total of 53 events detected correctly, out of 63 total events detected. (84.13%)
There was 76.81% events detected

Data from: LD04 with threshold of: 150 and min_consecutives of: 60
This data has 63 true events
There were 9077 frames correctly classified, out of a total of 10192 frames classified as interaction.
This results in an accuracy of 89.06%.
There were a total of 54 events detected correctly, out of 67 total events detected. (80.60%)
There was 85.71% events detected

Data from: LD13 with threshold of: 150 and min_consecutives of: 60
This data has 81 true events
There were 7301 frames correctly classified, out of a total of 9763 frames classified as interaction.
This results in an accuracy of 74.78%.
There were a total of 65 events 

Data from: LD03 with threshold of: 200 and min_consecutives of: 30
This data has 69 true events
There were 11989 frames correctly classified, out of a total of 18121 frames classified as interaction.
This results in an accuracy of 66.16%.
There were a total of 65 events detected correctly, out of 151 total events detected. (43.05%)
There was 94.20% events detected

Data from: LD04 with threshold of: 200 and min_consecutives of: 30
This data has 63 true events
There were 11891 frames correctly classified, out of a total of 16383 frames classified as interaction.
This results in an accuracy of 72.58%.
There were a total of 67 events detected correctly, out of 141 total events detected. (47.52%)
There was 106.35% events detected

Data from: LD13 with threshold of: 200 and min_consecutives of: 30
This data has 81 true events
There were 14296 frames correctly classified, out of a total of 27527 frames classified as interaction.
This results in an accuracy of 51.93%.
There were a total of 10

Data from: LD03 with threshold of: 225 and min_consecutives of: 45
This data has 69 true events
There were 12684 frames correctly classified, out of a total of 19275 frames classified as interaction.
This results in an accuracy of 65.81%.
There were a total of 64 events detected correctly, out of 135 total events detected. (47.41%)
There was 92.75% events detected

Data from: LD04 with threshold of: 225 and min_consecutives of: 45
This data has 63 true events
There were 12432 frames correctly classified, out of a total of 16680 frames classified as interaction.
This results in an accuracy of 74.53%.
There were a total of 63 events detected correctly, out of 116 total events detected. (54.31%)
There was 100.00% events detected

Data from: LD13 with threshold of: 225 and min_consecutives of: 45
This data has 81 true events
There were 15062 frames correctly classified, out of a total of 30233 frames classified as interaction.
This results in an accuracy of 49.82%.
There were a total of 86

Data from: LD03 with threshold of: 250 and min_consecutives of: 60
This data has 69 true events
There were 12454 frames correctly classified, out of a total of 20312 frames classified as interaction.
This results in an accuracy of 61.31%.
There were a total of 61 events detected correctly, out of 126 total events detected. (48.41%)
There was 88.41% events detected

Data from: LD04 with threshold of: 250 and min_consecutives of: 60
This data has 63 true events
There were 12827 frames correctly classified, out of a total of 17192 frames classified as interaction.
This results in an accuracy of 74.61%.
There were a total of 62 events detected correctly, out of 104 total events detected. (59.62%)
There was 98.41% events detected

Data from: LD13 with threshold of: 250 and min_consecutives of: 60
This data has 81 true events
There were 15784 frames correctly classified, out of a total of 32406 frames classified as interaction.
This results in an accuracy of 48.71%.
There were a total of 75 

In [6]:
interaction_counts = {}  # Create a dictionary to store interaction counts for each DataFrame

for data_frame_name, data_frame in result_dataframes.items():
    # Initialize variables to keep track of the current group
    current_group = None
    interaction_count = 0

    # Iterate through the "Behavior" column
    for behavior in data_frame["Behavior"]:
        if behavior == "interaction":
            if current_group != "interaction":
                # Start of a new interaction group
                interaction_count += 1
                current_group = "interaction"
        else:
            current_group = None  # Reset the group if behavior is not "interaction"

    interaction_counts[data_frame_name] = interaction_count

interaction_counts_60 = {}  # Create a dictionary to store interaction counts for each DataFrame

for data_frame_name, data_frame in result_dataframes.items():
    # Initialize variables to keep track of the current group
    current_group = None
    interaction_count = 0
    current_group_count = 0

    # Iterate through the "Behavior" column
    for behavior in data_frame["Behavior"]:
        if behavior == "interaction":
            if current_group != "interaction":
                # Start of a new interaction group
                current_group_count = 1
                current_group = "interaction"
            else:
                current_group_count += 1
        else:
            if current_group_count >= 60:
                interaction_count += 1
            current_group = None  # Reset the group if behavior is not "interaction"
            current_group_count = 0

    # Check if the last group, if any, was an "interaction" group and met the condition
    if current_group_count > 90:
        interaction_count += 1

    interaction_counts_60[data_frame_name] = interaction_count
    
print("all",interaction_counts)
print("+60",interaction_counts_60)

all {'LD03': 69, 'LD04': 63, 'LD13': 81, 'LD14': 34, 'LD23': 30, 'LD24': 50}
+60 {'LD03': 64, 'LD04': 57, 'LD13': 67, 'LD14': 29, 'LD23': 25, 'LD24': 42}


## Predicted Correct Events

In [7]:
# Define a list of threshold and min_consecutive values to test
threshold_values = [100, 125, 150, 175, 200, 225, 250]  # Add more values if needed
min_consecutive_values = [30, 45, 60]  # Add more values if needed

# Iterate through all data frames
for data_frame_name, data_frame in result_dataframes.items():
    # Initialize a nested dictionary to store correctly detected events
    correctly_detected_events = {}

    # Loop through threshold and min_consecutive values
    for threshold in threshold_values:
        correctly_detected_events[threshold] = {}  # Initialize inner dictionary
        for min_consecutive in min_consecutive_values:
            summary_values = {}

            # Initialize variables to keep track of the current group
            current_group = None
            interaction_count = 0

            # Iterate through the "Behavior" column
            for behavior in data_frame["Behavior"]:
                if behavior == "interaction":
                    if current_group != "interaction":
                        # Start of a new interaction group
                        interaction_count += 1
                        current_group = "interaction"
                else:
                    current_group = None  # Reset the group if behavior is not "interaction"

            summary_values[data_frame_name] = {"true interaction count": interaction_count}

            # Reset the other summary values to zero to avoid overwriting
            total_frames = 0
            correctly_detected_event = 0
            total = 0
            correctly_frame_count = 0

            interactions = detect_interactions(data_frame["Min_Distance"], min_consecutive, threshold)

            for interaction in interactions:
                start_frame, end_frame = interaction

                if end_frame - start_frame + 1 >= 15:
                    total += 1  # Increment the total count for eligible interactions
                    total_frames += end_frame - start_frame + 1
                    selected_data = data_frame.loc[start_frame:end_frame, "Behavior"]
                    mode_value = selected_data.mode().iloc[0]

                    if mode_value == "interaction" or mode_value == "TS":
                        correctly_detected_event += 1
                        correctly_frame_count += end_frame - start_frame + 1

                    # Store the prediction in the new column
                    data_frame.loc[start_frame:end_frame, "Interaction_Predictions"] = mode_value

                # Update the existing summary values in the summary_values dictionary
                summary_values[data_frame_name].update({
                    "total_frames": total_frames,
                    "correctly_detected_event": correctly_detected_event,
                    "total": total,
                    "correctly_frame_count": correctly_frame_count
                })

            correctly_detected_events[threshold][min_consecutive] = correctly_detected_event
    print(f"Data Frame: {data_frame_name}. With a total of: {interaction_count} events")  # Add a header for the data frame
    
    # Now, display the table for the current data frame
    print("\t" + "\t".join(map(str, threshold_values)))  # Print column headers
    for min_consecutive in min_consecutive_values:
        row_values = [str(min_consecutive)]  # Start the row with min_consecutive value
        for threshold in threshold_values:
            cell_value = correctly_detected_events[threshold][min_consecutive]
            row_values.append(str(cell_value))
        print("\t".join(row_values))

    # Add a line to separate data frames
    print("-" * 60)

Data Frame: LD03. With a total of: 69 events
	100	125	150	175	200	225	250
30	56	64	71	63	65	64	65
45	40	55	60	62	62	64	62
60	29	40	53	55	57	59	61
------------------------------------------------------------
Data Frame: LD04. With a total of: 63 events
	100	125	150	175	200	225	250
30	65	67	71	68	67	65	64
45	55	61	63	64	65	63	64
60	38	47	54	58	60	61	62
------------------------------------------------------------
Data Frame: LD13. With a total of: 81 events
	100	125	150	175	200	225	250
30	62	87	97	106	102	94	89
45	34	63	75	90	87	86	87
60	18	41	65	71	74	76	75
------------------------------------------------------------
Data Frame: LD14. With a total of: 34 events
	100	125	150	175	200	225	250
30	15	19	26	28	31	33	31
45	10	17	21	24	28	30	29
60	7	11	17	20	22	24	26
------------------------------------------------------------
Data Frame: LD23. With a total of: 30 events
	100	125	150	175	200	225	250
30	35	29	31	29	28	25	25
45	30	27	30	27	27	24	23
60	22	25	26	25	25	24	22
-------------------------

## Number of wrong Events wrongly detected

In [8]:
# Define a list of threshold and min_consecutive values to test
threshold_values = [100, 125, 150, 175, 200, 225, 250]  # Add more values if needed
min_consecutive_values = [30, 45, 60]  # Add more values if needed

# Iterate through all data frames
for data_frame_name, data_frame in result_dataframes.items():
    print(f"Data Frame: {data_frame_name}")  # Add a header for the data frame

    # Initialize a nested dictionary to store correctly detected events
    correctly_detected_events = {}

    # Loop through threshold and min_consecutive values
    for threshold in threshold_values:
        correctly_detected_events[threshold] = {}  # Initialize inner dictionary
        for min_consecutive in min_consecutive_values:
            summary_values = {}

            # Initialize variables to keep track of the current group
            current_group = None
            interaction_count = 0

            # Iterate through the "Behavior" column
            for behavior in data_frame["Behavior"]:
                if behavior == "interaction":
                    if current_group != "interaction":
                        # Start of a new interaction group
                        interaction_count += 1
                        current_group = "interaction"
                else:
                    current_group = None  # Reset the group if behavior is not "interaction"

            summary_values[data_frame_name] = {"true interaction count": interaction_count}
            
            # Initialize the summary values with default values
            summary_values[data_frame_name].update({
                "total_frames": 0,
                "correctly_detected_event": 0,
                "total": 0,
                "correctly_frame_count": 0,
                "total_detected_events": 0,
                "wrongly_detected_event": 0
            })

            # Reset the other summary values to zero to avoid overwriting
            total_frames = 0
            correctly_detected_event = 0
            total = 0
            correctly_frame_count = 0
            wrongly_detected_event = 0
            wrongly_detected_frames = 0

            interactions = detect_interactions(data_frame["Min_Distance"], min_consecutive, threshold)

            for interaction in interactions:
                start_frame, end_frame = interaction

                if end_frame - start_frame + 1 >= 15:
                    total += 1  # Increment the total count for eligible interactions
                    total_frames += end_frame - start_frame + 1
                    selected_data = data_frame.loc[start_frame:end_frame, "Behavior"]
                    mode_value = selected_data.mode().iloc[0]

                    if mode_value == "interaction" or mode_value == "TS":
                        correctly_detected_event += 1
                        correctly_frame_count += end_frame - start_frame + 1
                        
                    else:
                        wrongly_detected_event += 1
                        wrongly_detected_frames += end_frame - start_frame + 1
                        
                    # Store the prediction in the new column
                    data_frame.loc[start_frame:end_frame, "Interaction_Predictions"] = mode_value

                # Update the existing summary values in the summary_values dictionary
                summary_values[data_frame_name].update({
                    "total_frames": total_frames,
                    "correctly_detected_event": correctly_detected_event,
                    "total_detected_events": total,
                    "correctly_frame_count": correctly_frame_count,
                    "wrongly_detected_event": wrongly_detected_event,
                    "wrongly_detected_frames": wrongly_detected_frames
                })

            correctly_detected_events[threshold][min_consecutive] = summary_values[data_frame_name]["wrongly_detected_event"]  # Calculate and store the ratio

    # Now, display the table for the current data frame
    print("\t" + "\t".join(map(str, threshold_values)))  # Print column headers
    for min_consecutive in min_consecutive_values:
        row_values = [str(min_consecutive)]  # Start the row with min_consecutive value
        for threshold in threshold_values:
            cell_value = correctly_detected_events[threshold][min_consecutive]
            row_values.append(f"{cell_value}")  # Format as percentage
        print("\t".join(row_values))

    # Add a line to separate data frames
    print("-" * 60)

Data Frame: LD03
	100	125	150	175	200	225	250
30	12	21	38	62	86	112	133
45	8	10	14	30	52	71	87
60	4	6	10	17	28	51	65
------------------------------------------------------------
Data Frame: LD04
	100	125	150	175	200	225	250
30	17	27	38	50	74	98	133
45	11	11	18	29	39	53	73
60	7	10	13	19	28	35	42
------------------------------------------------------------
Data Frame: LD13
	100	125	150	175	200	225	250
30	28	62	106	134	177	204	219
45	9	23	51	84	119	140	168
60	4	10	27	45	73	97	121
------------------------------------------------------------
Data Frame: LD14
	100	125	150	175	200	225	250
30	1	3	7	16	22	30	53
45	0	1	1	3	8	10	20
60	0	1	1	1	2	4	6
------------------------------------------------------------
Data Frame: LD23
	100	125	150	175	200	225	250
30	15	20	20	19	22	23	25
45	10	13	13	12	18	16	20
60	7	9	11	7	11	11	17
------------------------------------------------------------
Data Frame: LD24
	100	125	150	175	200	225	250
30	13	27	57	82	103	112	122
45	2	15	25	45	62	84	98
60	1	3	14	22	32	54	65

In [9]:
###
###

## Number of correct frames

In [10]:
# Define a list of threshold and min_consecutive values to test
threshold_values = [100, 125, 150, 175, 200, 225, 250]  # Add more values if needed
min_consecutive_values = [30, 45, 60]  # Add more values if needed

# Iterate through all data frames
for data_frame_name, data_frame in result_dataframes.items():
    # Initialize a nested dictionary to store correctly detected frames
    correctly_detected_frames = {}

    # Loop through threshold and min_consecutive values
    for threshold in threshold_values:
        correctly_detected_frames[threshold] = {}  # Initialize inner dictionary
        for min_consecutive in min_consecutive_values:
            summary_values = {}

            # Initialize variables to keep track of the current group
            current_group = None
            interaction_count = 0

            # Iterate through the "Behavior" column
            for behavior in data_frame["Behavior"]:
                if behavior == "interaction":
                    if current_group != "interaction":
                        # Start of a new interaction group
                        interaction_count += 1
                        current_group = "interaction"
                else:
                    current_group = None  # Reset the group if behavior is not "interaction"

            summary_values[data_frame_name] = {"true interaction count": interaction_count}

            # Reset the other summary values to zero to avoid overwriting
            total_frames = 0
            correctly_detected_frames_count = 0  # Updated variable name
            total = 0
            correctly_frame_count = 0

            interactions = detect_interactions(data_frame["Min_Distance"], min_consecutive, threshold)

            for interaction in interactions:
                start_frame, end_frame = interaction

                if end_frame - start_frame + 1 >= 15:
                    total += 1  # Increment the total count for eligible interactions
                    total_frames += end_frame - start_frame + 1
                    selected_data = data_frame.loc[start_frame:end_frame, "Behavior"]
                    mode_value = selected_data.mode().iloc[0]

                    if mode_value == "interaction" or mode_value == "TS":
                        correctly_detected_frames_count += end_frame - start_frame + 1  # Updated variable name

                    # Store the prediction in the new column
                    data_frame.loc[start_frame:end_frame, "Interaction_Predictions"] = mode_value

                # Update the existing summary values in the summary_values dictionary
                summary_values[data_frame_name].update({
                    "total_frames": total_frames,
                    "correctly_detected_frames_count": correctly_detected_frames_count,  # Updated variable name
                    "total": total,
                    "correctly_frame_count": correctly_frame_count
                })

            correctly_detected_frames[threshold][min_consecutive] = correctly_detected_frames_count  # Updated variable name
    print(f"Data Frame: {data_frame_name}. With a total of: {data_frame['Behavior'].value_counts().get('interaction', 0)} frames")  # Add a header for the data frame

    # Now, display the table for the current data frame
    print("\t" + "\t".join(map(str, threshold_values)))  # Print column headers
    for min_consecutive in min_consecutive_values:
        row_values = [str(min_consecutive)]  # Start the row with min_consecutive value
        for threshold in threshold_values:
            cell_value = correctly_detected_frames[threshold][min_consecutive]  # Updated variable name
            row_values.append(str(cell_value))
        print("\t".join(row_values))

    # Add a line to separate data frames
    print("-" * 60)


Data Frame: LD03. With a total of: 13729 frames
	100	125	150	175	200	225	250
30	5281	7251	9226	10905	11989	12913	13171
45	4492	6708	8563	10567	11651	12684	12801
60	3745	5751	8016	9888	11166	12286	12454
------------------------------------------------------------
Data Frame: LD04. With a total of: 15986 frames
	100	125	150	175	200	225	250
30	7625	9030	10099	11028	11891	12892	13464
45	7001	8500	9735	10623	11453	12432	13171
60	6069	7712	9077	10154	11063	12260	12827
------------------------------------------------------------
Data Frame: LD13. With a total of: 20744 frames
	100	125	150	175	200	225	250
30	3849	6626	9554	12198	14296	15466	17036
45	2388	5453	8257	11080	13494	15062	16905
60	1533	3979	7301	9774	12108	14081	15784
------------------------------------------------------------
Data Frame: LD14. With a total of: 5423 frames
	100	125	150	175	200	225	250
30	1397	2196	2825	3442	4119	4522	4892
45	1218	2069	2641	3078	3902	4364	4771
60	1061	1689	2352	2873	3304	3834	4611
-------------------

## Number of wrong frames predicted

In [11]:
# Define a list of threshold and min_consecutive values to test
threshold_values = [100, 125, 150, 175, 200, 225, 250]  # Add more values if needed
min_consecutive_values = [30, 45, 60]  # Add more values if needed

# Iterate through all data frames
for data_frame_name, data_frame in result_dataframes.items():
    # Initialize a nested dictionary to store incorrectly detected frames
    incorrectly_detected_frames = {}

    # Loop through threshold and min_consecutive values
    for threshold in threshold_values:
        incorrectly_detected_frames[threshold] = {}  # Initialize inner dictionary
        for min_consecutive in min_consecutive_values:
            summary_values = {}

            # Initialize variables to keep track of the current group
            current_group = None
            interaction_count = 0

            # Iterate through the "Behavior" column
            for behavior in data_frame["Behavior"]:
                if behavior == "interaction":
                    if current_group != "interaction":
                        # Start of a new interaction group
                        interaction_count += 1
                        current_group = "interaction"
                else:
                    current_group = None  # Reset the group if behavior is not "interaction"

            summary_values[data_frame_name] = {"true interaction count": interaction_count}

            # Reset the other summary values to zero to avoid overwriting
            total_frames = 0
            correctly_detected_frames_count = 0
            incorrectly_detected_frames_count = 0  # Updated variable name
            total = 0
            correctly_frame_count = 0

            interactions = detect_interactions(data_frame["Min_Distance"], min_consecutive, threshold)

            for interaction in interactions:
                start_frame, end_frame = interaction

                if end_frame - start_frame + 1 >= 15:
                    total += 1  # Increment the total count for eligible interactions
                    total_frames += end_frame - start_frame + 1
                    selected_data = data_frame.loc[start_frame:end_frame, "Behavior"]
                    mode_value = selected_data.mode().iloc[0]

                    if mode_value == "interaction" or mode_value == "TS":
                        correctly_detected_frames_count += end_frame - start_frame + 1
                    else:
                        incorrectly_detected_frames_count += end_frame - start_frame + 1

                    # Store the prediction in the new column
                    data_frame.loc[start_frame:end_frame, "Interaction_Predictions"] = mode_value

                # Update the existing summary values in the summary_values dictionary
                summary_values[data_frame_name].update({
                    "total_frames": total_frames,
                    "correctly_detected_frames_count": correctly_detected_frames_count,
                    "incorrectly_detected_frames_count": incorrectly_detected_frames_count,  # Updated variable name
                    "total": total,
                    "correctly_frame_count": correctly_frame_count
                })

            incorrectly_detected_frames[threshold][min_consecutive] = incorrectly_detected_frames_count  # Updated variable name
    print(f"Data Frame: {data_frame_name}. With a total of: {data_frame['Behavior'].value_counts().get('interaction', 0)} frames")  # Add a header for the data frame

    # Now, display the table for the current data frame
    print("\t" + "\t".join(map(str, threshold_values)))  # Print column headers
    for min_consecutive in min_consecutive_values:
        row_values = [str(min_consecutive)]  # Start the row with min_consecutive value
        for threshold in threshold_values:
            cell_value = incorrectly_detected_frames[threshold][min_consecutive]  # Updated variable name
            row_values.append(str(cell_value))
        print("\t".join(row_values))

    # Add a line to separate data frames
    print("-" * 60)

Data Frame: LD03. With a total of: 13729 frames
	100	125	150	175	200	225	250
30	695	1193	2236	3660	6132	8505	11427
45	534	733	1164	2440	4360	6591	9241
60	336	531	901	1661	2850	5317	7858
------------------------------------------------------------
Data Frame: LD04. With a total of: 15986 frames
	100	125	150	175	200	225	250
30	954	1389	2102	3172	4492	6155	8788
45	750	835	1369	2426	3185	4248	6138
60	534	790	1115	1658	2632	3335	4365
------------------------------------------------------------
Data Frame: LD13. With a total of: 20744 frames
	100	125	150	175	200	225	250
30	1272	3070	5883	8829	13231	18126	22573
45	521	1623	3736	6788	10319	15171	19626
60	281	893	2462	4374	7722	11924	16622
------------------------------------------------------------
Data Frame: LD14. With a total of: 5423 frames
	100	125	150	175	200	225	250
30	40	129	284	637	939	1468	2471
45	0	61	69	168	456	619	1220
60	0	61	69	74	157	304	501
------------------------------------------------------------
Data Frame: LD23. With a t

#### Creates a dict with predicted | true labels for all dataframes with all param combinations

In [12]:
# Define a list of threshold and min_consecutive values to test
threshold_values = [100, 125, 150, 175, 200, 225, 250]
min_consecutive_values = [30, 45, 60]

# Initialize a dictionary to store results for each combination
results_dict = {}

# Iterate through all data frames
for data_frame_name, data_frame in result_dataframes.items():
    # Initialize a nested dictionary for the current data frame
    data_frame_results = {}

    # Loop through threshold and min_consecutive values
    for threshold in threshold_values:
        for min_consecutive in min_consecutive_values:
            interactions = detect_interactions(data_frame["Min_Distance"], min_consecutive, threshold)

            # Initialize an empty DataFrame to store "Behavior" and "Interaction_Predictions" columns
            selected_data = pd.DataFrame(columns=["Behavior", "Interaction_Predictions"])
            selected_data["Behavior"] = data_frame["Behavior"]
            selected_data["Interaction_Predictions"] = "background"

            for interaction in interactions:
                start_frame, end_frame = interaction

                if end_frame - start_frame + 1 >= 15:
                    selected_data.loc[start_frame:end_frame, "Interaction_Predictions"] = "interaction"

            # Store the selected data in the data frame results
            data_frame_results[(threshold, min_consecutive)] = selected_data

    # Store the data frame results in the overall results dictionary
    results_dict[data_frame_name] = data_frame_results

In [13]:
# Specify the folder where you want to save the CSV files
output_folder = r"C:\Users\raulo\Desktop\ze_vids\phase_3\results_dataframes\CleanerCentroid2_ClientAll_smaller_margin"

# Save the results to CSV files
for data_frame_name, data_frame_results in results_dict.items():
    for (threshold, min_consecutive), selected_data in data_frame_results.items():
        # Generate a filename based on the data frame name, threshold, and min_consecutive values
        filename = f"{data_frame_name}_threshold{threshold}_minc{min_consecutive}.csv"

        # Create the full path by joining the output folder and filename
        full_path = os.path.join(output_folder, filename)

        # Save the selected data to CSV
        selected_data.to_csv(full_path, index=False)

#### Creates plots with predicted | true horizontal histograms with 1000 frames each where it saves the plots

In [14]:
# import gc

# # Define the main output directory where you want to save the plots
# main_output_directory = r"C:\Users\raulo\Desktop\ze_vids\phase_3\Plots\smaller_distance-client-points_ALL_cleaner2points_W-Margin"

# # Create the main output directory if it doesn't exist
# os.makedirs(main_output_directory, exist_ok=True)

# # Iterate through data frames and their results
# for data_frame_name, data_frame_results in results_dict.items():
#     # Create a subdirectory for the current data frame within the main output directory
#     data_frame_directory = os.path.join(main_output_directory, data_frame_name)
#     os.makedirs(data_frame_directory, exist_ok=True)

#     for (threshold, min_consecutive), selected_data in data_frame_results.items():
#         # Create a subdirectory for the current parameter combination within the data frame directory
#         parameter_directory = os.path.join(data_frame_directory, f"T{threshold}_MC{min_consecutive}")
#         os.makedirs(parameter_directory, exist_ok=True)

#         # Define the true and predicted labels
#         true_labels = selected_data["Behavior"]
#         predicted_labels = selected_data["Interaction_Predictions"]

#         # Define the size of each chunk
#         chunk_size = 5000

#         # Calculate the number of chunks
#         num_chunks = len(true_labels) // chunk_size

#         for chunk in range(num_chunks):
#             start_frame = chunk * chunk_size
#             end_frame = start_frame + chunk_size

#             # Create a figure and axes for each chunk
#             fig, ax = plt.subplots()

#             # Create a horizontal bar for true labels
#             for frame, label in enumerate(true_labels[start_frame:end_frame]):
#                 color = 'blue' if label == "interaction" else 'gray'
#                 ax.barh(0, 1, left=frame + start_frame, color=color)

#             # Create a horizontal bar for predicted labels (below)
#             for frame, label in enumerate(predicted_labels[start_frame:end_frame]):
#                 color = 'blue' if label == "interaction" else 'gray'
#                 ax.barh(1, 1, left=frame + start_frame, color=color)

#             # Customize the plot
#             ax.set_yticks([0, 1])
#             ax.set_yticklabels(["True", "Predicted"])
#             ax.set_xlabel('Frame Number')
#             ax.set_xlim(start_frame, end_frame)
#             ax.set_ylim(-1, 2)

#             # Save the plot as an image in the parameter-specific subdirectory
#             output_file = os.path.join(parameter_directory, f"chunk{chunk}.png")
#             plt.savefig(output_file)
#             #plt.show()
#             # Close the current plot
#             plt.clf()
#             plt.close()
#             gc.collect()